**Librerias**

In [1]:
import warnings
warnings.simplefilter('ignore')

import selenium

import os
os.chmod('/Users/bryangarcia/Desktop/chromedriver', 755)

import time
import re
import sys
import requests
from pathlib import Path

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

**Funciones**

In [2]:
def get_page_source(url):
    """
    Esta es una funcion que extrae el codigo fuente de una pagina de Aliexpress
    
    Returns
        codigo fuente
    """
    randint = np.random.randint #Para evitar escribir todo el codigo
    
    #------------------------------#
    
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
#    options.add_argument('--headless') # se procesa sin mostrar la pantalla de Chrome
    
    driver = webdriver.Chrome(r'/Users/bryangarcia/Desktop/chromedriver',
                             chrome_options=options)
    
    driver.get(url)
    time.sleep(randint(3, high=7))
    ActionChains(driver).send_keys(Keys.END).perform() #Baja hasta el final de la pagina
    time.sleep(randint(3, high=7))
    ActionChains(driver).send_keys(Keys.END).perform()
    time.sleep(randint(3, high=7))
    
    page_source = driver.page_source
    driver.quit()
    
    return page_source


In [3]:
def get_links(page_source):
    """
    Recibe el codigo fuente de una pagina, extrae las urls y las regresa
    
    Returns
        list
        
    """
    starwars = BeautifulSoup(page_source, 'lxml')
    link_to_element = starwars.select('a.display-check-link')
    links = [link['href'] for link in link_to_element]
    links = list(dict.fromkeys(links))
    
    return links


In [4]:
def random_keypress_generator():
    keys = [Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_UP, Keys.ARROW_RIGHT, Keys.DOWN, Keys.END, Keys.PAGE_UP]
    action = np.random.choice(keys, p=[0.3, 0.05, 0.05, 0.1, 0.2, 0.2, 0.1])
    
    return action

In [5]:
def web_content(link):
    """
    Esta es una funcion que extrae el codigo fuente de una pagina de Aliexpress
    
    Returns
        codigo fuente
        
    """
    
    random = np.random.choice #Para evitar escribir todo el codigo
    
    #------------------------------#
    
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless') # se procesa sin mostrar la pantalla de Chrome
    
    driver = webdriver.Chrome(r'/Users/bryangarcia/Desktop/chromedriver',
                             chrome_options=options)
    
    driver.get(link)
    time.sleep(random(5))
    for ran in range(random(3)):
        ActionChains(driver).send_keys(random_keypress_generator()).perform()
        time.sleep(random(4))
    page_source = driver.page_source
    driver.quit()
    return page_source
    

In [6]:
def alies_parser(page_source):
    """
        Recibe una url y exrae cuatro atributos:
    
        -Nombre del elemeno
        -Descripcion
        -Afliacion
        -Locaciones
    
    Returns
    
    """
    element = BeautifulSoup(page_source, 'lxml')
    element_name = element.select('h3.title')
    element_name = element_name[0].get_text()
    
    element_desc = element.select('p.desc')
    element_desc = element_desc[0].get_text()
    element_desc
    
    element_afil = element.find_all('div', class_ = 'category', limit = 3)
    text = [x.text for x in element_afil]
    
    try:
        Affiliation = re.findall(r'(?<=Affiliations\s).+', text[1])
        Affiliation = r''.join(Affiliation)
        Affiliation = [e for e in re.split(r'\s*,\s*',Affiliation) if e]
        Affiliation = Affiliation[0].split()
        Affiliation = ' '.join(Affiliation)
        
        Location = re.findall(r'(?<=Locations\s).+', text[2])
        Location = r''.join(Location)
        Location = [e for e in re.split(r'\s*,\s*',Location) if e]
        Location = Location[0].split()
        Location = ' '.join(Location)
        
    except IndexError:
        Affiliation = ""
        Location = ""
    
    
    return element_name, element_desc, Affiliation, Location

In [7]:
def alis_parson_project():
    
    url = 'https://www.starwars.com/databank'
    page_source = get_page_source(url)
    links = get_links(page_source)
    print(len(links))

    for num, link in enumerate(links, start=1):

        print(f"This link is #{num}")
        filename = f"element_{num}_stawars_databank.csv"
        filename = "data_starwars/"+ filename
        print("Document will be place and call: "+ filename)

        if not Path(filename).is_file():
            page_source = web_content(link)
            print(link)
            print("Finish extracting product's page source.")
            print("Parsing content.")
            variables = alies_parser(page_source)
            print("Finish Parsing. Appending.")
            variables = list(variables) + [link]
            print(variables)
            df = pd.DataFrame([variables], columns=["Element_Name", "Description",
                                     "Affiliation", "Location", 
                                     "Link"],
                          dtype=object)

            df.to_csv(filename, index = False)
            print(f"{filename} saved.")

    print("Finished!")

**Scrapping**

In [8]:
alis_parson_project()

79
This link is #1
Document will be place and call: data_starwars/element_1_stawars_databank.csv
This link is #2
Document will be place and call: data_starwars/element_2_stawars_databank.csv
https://www.starwars.com/databank/stormtroopers
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' stormtroopers ', 'Stormtroopers are elite shock troops fanatically loyal to the Empire and impossible to sway from the Imperial cause. They wear imposing white armor, which offers a wide range of survival equipment and temperature controls to allow the soldiers to survive in almost any environment. Stormtroopers wield blaster rifles and pistols with great skill, and attack in hordes to overwhelm their enemies. Along with standard stormtroopers, the Empire has organized several specialized units, including snowtroopers and scout troopers.', '', '', 'https://www.starwars.com/databank/stormtroopers']
data_starwars/element_2_stawars_databank.csv saved.
This link is #3


https://www.starwars.com/databank/t-16-skyhopper-model
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' T-16 skyhopper model  ', 'Purchased from the Jawa scavenger Teeka, this T-16 skyhopper model needs a little repair and cleaning before Obi-Wan Kenobi can give it as a gift to young Luke Skywalker.', '', '', 'https://www.starwars.com/databank/t-16-skyhopper-model']
data_starwars/element_13_stawars_databank.csv saved.
This link is #14
Document will be place and call: data_starwars/element_14_stawars_databank.csv
https://www.starwars.com/databank/the-lars-homestead
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' the Lars homestead  ', "Two generations of the Lars family sought to make a living on this hardscrabble farm on Tatooine, spending decades in the unforgiving climate coaxing moisture from the sandy planet's arid skies. Anakin Skywalker visited the farm in search of his mother Shmi, who’d married Clieg

https://www.starwars.com/databank/the-twins
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' The Twins ', 'Cousins to the late, great Jabba the Hutt, a pair of fraternal Huttese siblings, referred to simply as the Twins, have designs for the criminal empire they see as their birthright. Overcoming their limited mobility and obvious avarice by being carried aloft on enormous litters, they are ferried along the streets by a legion of devoted servants, including a drummer who announces their arrival and a gladiator who stands guard. ', '', '', 'https://www.starwars.com/databank/the-twins']
data_starwars/element_27_stawars_databank.csv saved.
This link is #28
Document will be place and call: data_starwars/element_28_stawars_databank.csv
https://www.starwars.com/databank/wed-treadwell-droid
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' WED Treadwell Droid ', 'WED Treadwell droids are found throughout the galaxy

https://www.starwars.com/databank/aat-battle-tank
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' AAT Battle Tank ', 'The marriage of repulsorlift technology and heavy armor has resulted in formidable floating tanks known as AATs. These assault tanks form the frontline of Trade Federation armored infantry divisions, as well as Separatist forces during the Clone Wars. A crew of battle droids drives the battle tank and operates the array of laser and projectile weaponry carried by the vehicle.', '', '', 'https://www.starwars.com/databank/aat-battle-tank']
data_starwars/element_42_stawars_databank.csv saved.
This link is #43
Document will be place and call: data_starwars/element_43_stawars_databank.csv
https://www.starwars.com/databank/aiwha
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' Aiwha ', 'These massive cetaceans were native to the oceans of Kamino, swimming beneath the waves and sometimes leaping out

https://www.starwars.com/databank/clink
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' Clink ', "An astromech with no patience for impropriety, Clink does not suffer fools or the interruption of orderly function in the store room of a trader's post.", '', '', 'https://www.starwars.com/databank/clink']
data_starwars/element_53_stawars_databank.csv saved.
This link is #54
Document will be place and call: data_starwars/element_54_stawars_databank.csv
https://www.starwars.com/databank/gs-8
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' GS-8 ', 'GS-8 is a loyal protocol droid in the service of Senator Avi Singh on the planet Raxus. Even after her master is arrested, GS-8 can be counted on to execute his instructions to the letter, as per her programming.', '', '', 'https://www.starwars.com/databank/gs-8']
data_starwars/element_54_stawars_databank.csv saved.
This link is #55
Document will be place and call: dat

https://www.starwars.com/databank/4d-m1n
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' 4D-M1N ', 'When Captain Doza needs guests escorted into Doza Tower or messages relayed to citizens of the Colossus platform, he calls upon his loyal droid 4D-M1N to act as liaison, rarely leaving the gleaming corridors surrounding his private office and living quarters. ', '', '', 'https://www.starwars.com/databank/4d-m1n']
data_starwars/element_67_stawars_databank.csv saved.
This link is #68
Document will be place and call: data_starwars/element_68_stawars_databank.csv
https://www.starwars.com/databank/5-l
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' 5-L ', 'A mouse droid serving aboard a First Order Star Destroyer, 5-L is briefly reprogrammed to serve Tam Ryvora on a covert mission to return to her friends aboard the Colossus.', '', '', 'https://www.starwars.com/databank/5-l']
data_starwars/element_68_stawars_datab

https://www.starwars.com/databank/abednedo
Finish extracting product's page source.
Parsing content.
Finish Parsing. Appending.
[' Abednedo ', 'A gregarious and clever species, Abednedos are common sights in the galaxy, pursuing a range of professions on many planets. They hail from a planet of the same name, which is noted for its sprawling, colorful and chaotic cities. After the Battle of Endor, the Empire attacked Abednedo with a climate-disruption array; as a result, many Abednedos support the New Republic or the Resistance. Their native language is Abednedish, though many speak Basic.', '', '', 'https://www.starwars.com/databank/abednedo']
data_starwars/element_79_stawars_databank.csv saved.
Finished!
